*Eduardus Tjitrahardja | @edutjie | 2022*

# Import Libraries


In [28]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.special
from IPython.display import Image

# Neural Network class definition

### The Skeleton Code

- initialisation - to set the number of input, hidden and output nodes
- train - refine the weights after being given a training set example to learn from
- query - give an answer from the output nodes after being given an input


In [29]:
class NeuralNetwork:
    # initialise the neural network
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        self.inodes = input_nodes
        self.hnodes = hidden_nodes
        self.onodes = output_nodes
        self.lr = learning_rate

        # generate weights between input and hidden layers
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc
        self.wih = np.random.normal(
            0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes)
        )
        self.who = np.random.normal(
            0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes)
        )

        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)

        # update weight function
        self.update_weight = lambda Ek, Ok, Oj: self.lr * np.dot(
            Ek * Ok * (1 - Ok), Oj.T
        )

    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin=2).T
        # convert targets list to 2d array
        targets = np.array(targets_list, ndmin=2).T

        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)

        # calculate signals into final output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)

        # output = target - actual
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = np.dot(self.who.T, output_errors)

        # update the weights for the links between the hidden and output layers
        self.who += self.update_weight(output_errors, final_outputs, hidden_outputs)
        # update the weights for the links between the input and hidden layers
        self.wih += self.update_weight(hidden_errors, hidden_outputs, inputs)

    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin=2).T

        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)

        # calculate signals into final output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)

        return final_outputs


### Create Instance


In [30]:
# set the number of each node in the neural network and its learning rate
input_nodes, hidden_nodes, output_nodes, learning_rate = 3, 3, 3, 0.3

# create neural network instance
n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)


## Class definition (step by step)


### `__init__` function

#### Generate Weights


In [31]:
np.random.rand(n.hnodes, n.inodes) - 0.5


array([[-0.06533014,  0.49523657, -0.2145751 ],
       [ 0.29808156, -0.25753135,  0.47186705],
       [ 0.30891868,  0.3793002 ,  0.24739927]])

**Optional**:

- 0.0 is the center of the normal distribution
- pow(n.hnodes, -0.5) is the standart deviation
- (n.hnodes, n.inodes) is the matrix size


In [32]:
# optional
np.random.normal(0.0, pow(n.hnodes, -0.5), (n.hnodes, n.inodes))


array([[-0.75534646,  0.73907531,  0.16509461],
       [-0.02321928, -0.12865966, -0.80537724],
       [-0.27581861,  0.09091621,  0.57888023]])

### `query` Function

Xhidden = Winput_hidden · I


In [33]:
inputs_list = np.random.rand(1, 3)
inputs = np.array(inputs_list, ndmin=2).T
inputs


array([[0.80960127],
       [0.82368742],
       [0.6907873 ]])

In [34]:
hidden_inputs = np.dot(n.wih, inputs)


expit() from scipy.special is used to calculate the sigmoid function

> `activation_function = lambda x: scipy.special.expit(x)`


In [35]:
# calculate the signals emerging from hidden layer
hidden_output = n.activation_function(hidden_inputs)


### Testing progress so far


In [36]:
n.query([1, 0.5, -1.5])


array([[0.56597283],
       [0.50336555],
       [0.44884885]])

### `train` Function


- started with the same code as `query()`
- _error = target - actual_
  - > `output_errors = targets - final_outputs`
- _errors_hidden = weights^T_hidden_output · errors_output_
  - > `hidden_errors = numpy.dot(self.who.T, output_errors)`
- Formula to update the weights:


In [38]:
Image(url="images\deltaW_formula.png")


- convert to code:
  - > `deltaW_j_k = self.lr * np.dot( output_errors * final_outputs * (1 - final_outputs), hidden_output.T )`
